TripAdvisor European Restaurants


In [1]:
# Import
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import pycountry


df = pd.read_csv("dataset/tripadvisor_european_restaurants.csv")
df

/var/folders/58/1r1s059j6m18hjjrmd4ypz200000gn/T/ipykernel_89921/4043934525.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset/tripadvisor_european_restaurants.csv")


,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,...,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,...,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,NaN
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,...,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,g10002858-d4586832,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133,...,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,g10002986-d3510044,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460,...,1.0,0.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN
4,g10022428-d9767191,Relais Du MontSeigne,"[""Europe"", ""France"", ""Occitanie"", ""Aveyron"", ""...",France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,"route du Montseigne, 12620 Saint-Laurent-de-Le...",44.208860,2.960470,...,4.0,7.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083392,g9710275-d10770782,Complex Popas Pacurari,"[""Europe"", ""Romania"", ""Northeast Romania"", ""Ia...",Romania,Northeast Romania,Iasi County,NaN,"Soseaua Pacurari, Valea Lupului 707410 Romania",47.172950,27.519110,...,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1083393,g9716321-d15026574,Casa Pastravarului DORIPESCO,"[""Europe"", ""Romania"", ""Transylvania"", ""Central...",Romania,Transylvania,Brasov County,Apata,"DN 13 Judetul Kilometrul 33 Maierus, Apata 507...",45.904423,25.470509,...,0.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
1083394,g9722813-d15891057,Hanul Tentea,"[""Europe"", ""Romania"", ""Transylvania"", ""Northwe...",Romania,Transylvania,Maramures County,Sacel,"DN17C, Sacel Romania",47.631920,24.450910,...,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
1083395,g9726871-d21391722,Casa Paduraru,"[""Europe"", ""Romania"", ""Southern Romania"", ""Arg...",Romania,Southern Romania,Arges County,NaN,"Sat. Argeselu Numarul 432, Maracineni 117450 R...",44.918950,24.867634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
price_mapping = {"€": 1, "€€": 2, "€€€": 3, "€€€€": 4}
df["price_level"] = df["price_level"].map(price_mapping)
def mean_price(row):
    if pd.isna(row):
        return np.nan
    try:
        prices = row.replace("€", "").split("-")
        prices = [float(p) for p in prices if p]
        if len(prices) == 2:
            return np.mean(prices)
        elif len(prices) == 1:
            return prices[0]
    except:
        return np.nan

df["mean_price"] = df["price_range"].apply(mean_price)
df[["price_range", "mean_price"]].head()
df["mean_price"].head(10)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5    21.5
6    12.5
7    22.5
8    19.0
9    21.0
Name: mean_price, dtype: float64

In [3]:
import plotly.express as px

# Bar plot: Percentage of restaurants per country (grouping countries with <10k restaurants as "Others" and placing 'Others' at the bottom, highest on top)
country_counts = df["country"].value_counts().reset_index()
country_counts.columns = ["country", "restaurant_count"]

# Mark countries with less than 10,000 restaurants as "Others"
country_counts["country_grouped"] = country_counts["country"]
country_counts.loc[country_counts["restaurant_count"] < 10000, "country_grouped"] = "Others"

# Group again to sum "Others"
grouped_counts = country_counts.groupby("country_grouped")["restaurant_count"].sum().reset_index()

# Calculate percentage
total_restaurants = grouped_counts["restaurant_count"].sum()
grouped_counts["percentage"] = (grouped_counts["restaurant_count"] / total_restaurants) * 100

# Sort by restaurant_count descending, but put 'Others' at the bottom
main_rows = grouped_counts[grouped_counts["country_grouped"] != "Others"].sort_values("restaurant_count", ascending=False)
others_row = grouped_counts[grouped_counts["country_grouped"] == "Others"]
grouped_counts = pd.concat([main_rows, others_row], ignore_index=True)

# Assign colors: light orange for 'Others', blue for the rest
colors = ['#1f77b4'] * len(grouped_counts)
if 'Others' in grouped_counts["country_grouped"].values:
    idx = grouped_counts[grouped_counts["country_grouped"] == "Others"].index[0]
    colors[idx] = '#FFB347'  # light orange

fig = px.bar(
    grouped_counts,
    x="percentage",
    y="country_grouped",
    title="Percentage of Restaurants per Country",
    labels={"country_grouped": "Country", "percentage": "Percentage of Restaurants (%)"},
    text=grouped_counts["percentage"].map("{:.2f}%".format),
    height=800,
)
fig.update_traces(texttemplate='%{text}', textposition='outside', marker_color=colors)
fig.update_layout(
    yaxis={'categoryorder': 'array', 'categoryarray': grouped_counts["country_grouped"].tolist()[::-1]},
    font=dict(size=18), 
    title_font=dict(size=24),
)
fig.show()

In [4]:
# Calculate the average price per country (only countries with >10k restaurants)
country_counts = df["country"].value_counts()
countries_over_10k = country_counts[country_counts > 10000].index

df_avg_price = df[df["country"].isin(countries_over_10k)].groupby("country")["mean_price"].mean().reset_index()
df_avg_price.columns = ["country", "mean_price"]
df_avg_price = df_avg_price.sort_values("mean_price", ascending=True)

fig = px.bar(
    df_avg_price,
    x="mean_price",
    y="country",
    title="Average price per country (in €)",
    labels={"mean_price": "Average Price Range", "country": "Country"},
    text="mean_price",
    height=800
)


fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    font=dict(size=18), 
    title_font=dict(size=24),
)
fig.show()

In [5]:
# Add these replacements before mapping ISO codes
df["country"] = df["country"].replace({
    "The Netherlands": "Netherlands",
    "Great Britain": "United Kingdom",
    "Scotland": "United Kingdom"
})

# Calculate the average price per country (all countries)
df_avg_price = df.groupby("country")["mean_price"].mean().reset_index()
df_avg_price.columns = ["country", "mean_price"]

# Map country names to ISO Alpha-3 codes
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except:
        return None

df_avg_price["iso_alpha"] = df_avg_price["country"].apply(get_iso3)
df_avg_price = df_avg_price.dropna(subset=["iso_alpha"])

fig = px.choropleth(
    df_avg_price,
    locations="iso_alpha",
    color="mean_price",
    hover_name="country",
    color_continuous_scale="Viridis",
    title="Average Restaurant Price per Country in Europe",
    labels={"mean_price": "Average Price (€)"},
    scope="europe",  # Focus map on Europe
    projection="natural earth",
    fitbounds="locations",
    width=800,
    height=600,
)
fig.update_geos(
    showcountries=True,
    lataxis_range=[35, 72],
    lonaxis_range=[-25, 45]
)
fig.update_layout(
    font=dict(size=18),
    title_font=dict(size=24),
)
fig.show()

In [6]:
# Calculate the average rating per country (all countries)
df_avg_rating = df.groupby("country")["avg_rating"].mean().reset_index()
df_avg_rating.columns = ["country", "avg_rating"]

# Standardize country names for mapping
df_avg_rating["country"] = df_avg_rating["country"].replace({
    "The Netherlands": "Netherlands",
    "Great Britain": "United Kingdom",
    "Scotland": "United Kingdom"
})

# Map country names to ISO Alpha-3 codes
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except:
        return None

df_avg_rating["iso_alpha"] = df_avg_rating["country"].apply(get_iso3)
df_avg_rating = df_avg_rating.dropna(subset=["iso_alpha"])

fig = px.choropleth(
    df_avg_rating,
    locations="iso_alpha",
    color="avg_rating",
    hover_name="country",
    color_continuous_scale="Viridis",
    title="Average Restaurant Rating per Country in Europe",
    labels={"avg_rating": "Average Rating"},
    scope="europe",
    projection="natural earth",
    fitbounds="locations",
    width=800,
    height=600,
)
fig.update_geos(
    showcountries=True,
    lataxis_range=[35, 72],
    lonaxis_range=[-25, 45]
)
fig.update_layout(
    font=dict(size=18),
    title_font=dict(size=24),
)
fig.show()

In [7]:
# Cuisine vs. mean price: Heatmap with number of restaurants in each spot

# Explode cuisines if stored as a comma-separated string
df_exploded = df.copy()
df_exploded["cuisines"] = df_exploded["cuisines"].str.split(", ")
df_exploded = df_exploded.explode("cuisines")

# Bin mean_price (e.g., bins of 5€ from 0 to 35)
bins = range(0, 36, 5)
df_exploded["mean_price_bin"] = pd.cut(df_exploded["mean_price"], bins=bins, right=False)

# Only keep top 15 cuisines for readability
top_cuisines = df_exploded["cuisines"].value_counts().head(15).index
df_heatmap = df_exploded[df_exploded["cuisines"].isin(top_cuisines)]

# Group by cuisine and mean_price_bin, count restaurants
heatmap_data = df_heatmap.groupby(["cuisines", "mean_price_bin"]).size().reset_index(name="count")

# Pivot for heatmap
heatmap_pivot = heatmap_data.pivot(index="cuisines", columns="mean_price_bin", values="count").fillna(0)

# Plot heatmap
import plotly.express as px
fig = px.imshow(
    heatmap_pivot,
    labels=dict(x="Mean Price Range (€)", y="Cuisine", color="Number of Restaurants"),
    x=[f"{interval.left:.0f}–{interval.right-1:.0f}" for interval in heatmap_pivot.columns],
    y=heatmap_pivot.index,
    color_continuous_scale="Viridis",
    title="Cuisine vs. Mean Price Range (Number of Restaurants)",
    aspect="auto",
)
fig.update_layout(
    font=dict(size=16),
    title_font=dict(size=22),
    height=600,
    width=1200,
    xaxis_tickfont=dict(size=18)
)
fig.show()

/var/folders/58/1r1s059j6m18hjjrmd4ypz200000gn/T/ipykernel_89921/727753936.py:17: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [8]:
# Bar chart: Mean price for each cuisine (top 15 cuisines)

# Explode cuisines if stored as a comma-separated string
df_exploded = df.copy()
df_exploded["cuisines"] = df_exploded["cuisines"].str.split(", ")
df_exploded = df_exploded.explode("cuisines")

# Only keep top 15 cuisines for readability
top_cuisines = df_exploded["cuisines"].value_counts().head(15).index
df_bar = df_exploded[df_exploded["cuisines"].isin(top_cuisines)]

# Calculate mean price for each cuisine
cuisine_mean_price = df_bar.groupby("cuisines")["mean_price"].mean().reset_index()

# Sort by mean price
cuisine_mean_price = cuisine_mean_price.sort_values("mean_price", ascending=True)

# Plot bar chart
import plotly.express as px
fig = px.bar(
    cuisine_mean_price,
    x="mean_price",
    y="cuisines",
    orientation="h",
    title="Mean Price for Each Cuisine (Top 15 Cuisines)",
    labels={"mean_price": "Mean Price (€)", "cuisines": "Cuisine"},
    text="mean_price"
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    font=dict(size=16),
    title_font=dict(size=22),
    height=600
)
fig.show()

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Heatmap and Barplot data ---
df_exploded = df.copy()
df_exploded["cuisines"] = df_exploded["cuisines"].str.split(", ")
df_exploded = df_exploded.explode("cuisines")
bins = range(0, 36, 5)
df_exploded["mean_price_bin"] = pd.cut(df_exploded["mean_price"], bins=bins, right=False)
top_cuisines = df_exploded["cuisines"].value_counts().head(15).index
df_heatmap = df_exploded[df_exploded["cuisines"].isin(top_cuisines)]

# Calculate mean price for each cuisine and sort
cuisine_mean_price = df_heatmap.groupby("cuisines")["mean_price"].mean().reset_index()
cuisine_mean_price = cuisine_mean_price.sort_values("mean_price", ascending=True)
ordered_cuisines = cuisine_mean_price["cuisines"]

# Prepare heatmap data with sorted cuisines
heatmap_data = df_heatmap.groupby(["cuisines", "mean_price_bin"]).size().reset_index(name="count")
heatmap_pivot = heatmap_data.pivot(index="cuisines", columns="mean_price_bin", values="count").fillna(0)
heatmap_pivot = heatmap_pivot.loc[ordered_cuisines]  # reorder index

heatmap_x = [f"{interval.left:.0f}–{interval.right-1:.0f}" for interval in heatmap_pivot.columns]
heatmap_y = heatmap_pivot.index
heatmap_z = heatmap_pivot.values

# Barplot data (already sorted)
bar_x = cuisine_mean_price["mean_price"]
bar_y = cuisine_mean_price["cuisines"]

# --- Create subplots ---
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    subplot_titles=("Cuisine vs. Mean Price Range (Number of Restaurants)", "Mean Price for Each Cuisine"),
    horizontal_spacing=0.08
)

# Heatmap (left)
fig.add_trace(
    go.Heatmap(
        z=heatmap_z,
        x=heatmap_x,
        y=heatmap_y,
        colorscale="Viridis",
        colorbar=dict(title="Number of Restaurants"),
        hovertemplate="Cuisine: %{y}<br>Price: %{x}<br>Count: %{z}<extra></extra>",
    ),
    row=1, col=1
)

# Bar chart (right)
fig.add_trace(
    go.Bar(
        x=bar_x,
        y=bar_y,
        orientation="h",
        marker_color="blue",
        showlegend=False
    ),
    row=1, col=2
)

# Layout
fig.update_xaxes(title_text="Mean Price Range (€)", row=1, col=1, tickangle=-45)
fig.update_yaxes(title_text="Cuisine", row=1, col=1)
fig.update_xaxes(title_text="Mean Price (€)", row=1, col=2)
fig.update_yaxes(showticklabels=False, row=1, col=2)

fig.update_layout(
    font=dict(size=16),
    title_font=dict(size=22),
    height=700,
    width=1200,
    margin=dict(t=80),
)
fig.show()

/var/folders/58/1r1s059j6m18hjjrmd4ypz200000gn/T/ipykernel_89921/2291701861.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [10]:
import pandas as pd
import plotly.express as px

# Conta tutti i ristoranti per paese
ristoranti_totali = df['country'].value_counts().reset_index()
ristoranti_totali.columns = ['country', 'ristoranti_totali']

# Filtra i ristoranti aperti 7 giorni su 7
df_7gg = df[df['open_days_per_week'] == 7]
ristoranti_7gg = df_7gg['country'].value_counts().reset_index()
ristoranti_7gg.columns = ['country', 'ristoranti_7gg']

# Unisci i due conteggi
df_merge = pd.merge(ristoranti_totali, ristoranti_7gg, on='country', how='left')
df_merge['ristoranti_7gg'] = df_merge['ristoranti_7gg'].fillna(0)

# Calcola la percentuale
df_merge['percentuale_7gg'] = (df_merge['ristoranti_7gg'] / df_merge['ristoranti_totali']) * 100

# Mappa ISO dei paesi
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except:
        return None

df_merge['iso_alpha'] = df_merge['country'].apply(get_iso3)
df_grouped = df_merge.groupby('iso_alpha', as_index=False).agg({'percentuale_7gg': 'mean'})

# Mappa coropletica (scala gradevole: 'YlGnBu')
fig = px.choropleth(
    df_grouped,
    locations='iso_alpha',
    color='percentuale_7gg',
    color_continuous_scale='Viridis',
    title='Percentuale di ristoranti aperti 7 giorni su 7 per paese',
    labels={'percentuale_7gg': '% Ristoranti 7/7'},
    width=800,
    height=600,
    scope="europe",
    projection="natural earth",
    fitbounds="locations"
)

fig.update_geos(
    showcountries=True,
    lataxis_range=[35, 72],
    lonaxis_range=[-25, 45]
)

fig.update_layout(
    coloraxis_colorbar=dict(title="% Ristoranti 7/7"),
    font=dict(size=18),
    title_font=dict(size=24),
)
fig.show()

In [11]:
# List of famous fast food names to exclude (add more as needed)
exclude_names = [
    "McDonald's", "Burger King", "KFC", "Subway", "Starbucks", "Domino's", "Pizza Hut",
    "Papa John's", "Five Guys", "Dunkin'", "Wendy's", "Taco Bell", "Hard Rock Cafe",
    "Domino's Pizza", "Costa Coffee", "Wild Bean Cafe", "BP","Telepizza", "La Piadineria",
    "Old Wild West", "Tesco Cafe", "Buffalo Grill", "Caffe Nero", "Pret a Manger",
    "Papa John's Pizza", "Leon", "Nando's", "Wimpy", "Greggs", "Caffe Ritazza",
    "Boulangerie Marie Blachere", "Pizza Express", "Pizza Hut Delivery", "Columbus Cafe & Co",
    "100 Montaditos", "Coffee Island", "Courtepaille"
]

# Count restaurant names, excluding fast food chains
name_counts = (
    df[~df["restaurant_name"].str.lower().isin([n.lower() for n in exclude_names])]
    ["restaurant_name"]
    .value_counts()
    .head(20)
    .reset_index()
)
name_counts.columns = ["name", "count"]

# Bar plot of the most used restaurant names (excluding fast food chains)
fig = px.bar(
    name_counts,
    x="count",
    y="name",
    orientation="h",
    title="Most Used Restaurant Names (Excluding Famous Fast Food Chains)",
    labels={"count": "Number of Restaurants", "name": "Restaurant Name"},
    height=700,
)
fig.update_layout(
    font=dict(size=16),
    title_font=dict(size=22),
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

In [13]:
from geopy.distance import geodesic

# Coordinates for Rome city centre (e.g., Colosseo)
rome_center = (41.8902, 12.4922)

# Filter for Rome
df_rome = df[df['city'].str.lower() == 'rome'].copy()

# Compute distance from city centre for each restaurant
def compute_distance(row):
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        return None
    return geodesic((row['latitude'], row['longitude']), rome_center).km

df_rome['distance_km'] = df_rome.apply(compute_distance, axis=1)

# Define distance bands (e.g., 0-1km, 1-2km, ..., 5+km)
bins = [0, 1, 2, 3, 4, 5, float('inf')]
labels = ['0–1km', '1–2km', '2–3km', '3–4km', '4–5km', '5km+']
df_rome['distance_band'] = pd.cut(df_rome['distance_km'], bins=bins, labels=labels, right=False)

# Calculate average price per band
avg_price_band = df_rome.groupby('distance_band')['mean_price'].mean().reset_index()

# Plot average price by distance band
fig = px.bar(
    avg_price_band,
    x='distance_band',
    y='mean_price',
    title='Average Restaurant Price in Rome by Distance from City Centre',
    labels={'distance_band': 'Distance from Centre', 'mean_price': 'Average Price (€)'},
    text='mean_price'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(font=dict(size=16), title_font=dict(size=22))
fig.show()

ModuleNotFoundError: No module named 'geopy'